In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
from grouper import group_by_volume, group_by_time
from pyfolio.timeseries import perf_stats
from functools import partial
from utils import chart_price, plot, v_backtester, c_backtester, perf, perf_var, breakout_strategy, bootstrap, m_proc
import sys
sys.path.append('/home/tomek/ib_tools')
from indicators import get_ATR, get_signals
import matplotlib.pyplot as plt
from typing import NamedTuple
from collections import namedtuple
%matplotlib inline

In [2]:
from datastore_pytables import Store

In [3]:
store = Store()
store.keys()

['/cont/min/CL_20191120_NYMEX_USD',
 '/cont/min/ES_20191220_GLOBEX_USD',
 '/cont/min/GC_20191227_NYMEX_USD',
 '/cont/min/GE_20191216_GLOBEX_USD',
 '/cont/min/NKD_20191212_GLOBEX_USD',
 '/cont/min/NQ_20191220_GLOBEX_USD',
 '/cont/min/YM_20191220_ECBOT_USD',
 '/cont/min/ZB_20191219_ECBOT_USD',
 '/cont/min/ZF_20191231_ECBOT_USD',
 '/cont/min/ZN_20191219_ECBOT_USD']

In [4]:
contract = store.read('/cont/min/CL_20191120_NYMEX_USD').sort_index()

In [5]:
table = bootstrap(contract,
          start = '20180101', 
          end = '20181231',
          paths=100)

In [6]:
func = partial(breakout_strategy, 
                  time_int = 45,
                  periods = [5, 10, 20, 40, 80, 160],
                  ema_fast = 30,
                  ema_slow = 120,
                  atr_periods = 180,
                  sl_atr = 3,
                  take_profit=0
                  )

In [7]:
results = m_proc(table, func)

In [8]:
res = pd.DataFrame([perf(i, output=False).stats for i in results])

In [9]:
res[res['Annual return'] > 0]['Annual return'].count()

79

In [10]:
res.sort_values('Annual return')

,Annual return,Cumulative returns,Annual volatility,Sharpe ratio,Calmar ratio,Stability,Max drawdown,Omega ratio,Sortino ratio,Skew,...,Avg gain/loss ratio,Position EV,Positions per day,Days per position,Actual avg. duration,Days,Positions,Trades,Monthly EV,Annual EV
27,-0.466511,-0.350365,0.453822,-1.160398,-0.873142,0.800025,-0.534290,0.808294,-1.754611,0.952520,...,1.983487,-0.168554,0.728324,1.373016,0 days 22:55:00,173,126,252,-2.578006,-30.936067
84,-0.330504,-0.239558,0.254941,-1.446472,-0.873618,0.849942,-0.378317,0.772607,-2.047217,0.666374,...,1.486656,-0.116708,0.726744,1.376000,0 days 23:09:00,172,125,249,-1.781147,-21.373764
29,-0.329715,-0.238947,0.273554,-1.325904,-1.081668,0.880276,-0.304821,0.789570,-1.900676,0.579474,...,1.210941,-0.113001,0.732558,1.365079,1 days 00:10:00,172,126,252,-1.738382,-20.860588
2,-0.272803,-0.194397,0.223887,-1.311652,-1.241597,0.025792,-0.219720,0.798971,-1.993206,1.120141,...,1.975701,-0.096698,0.707602,1.413223,0 days 23:45:00,171,121,242,-1.436894,-17.242732
81,-0.254055,-0.181314,0.219856,-1.225049,-1.095450,0.693724,-0.231919,0.785619,-2.009468,2.184753,...,1.671103,-0.093097,0.709302,1.409836,0 days 23:35:00,172,122,244,-1.386713,-16.640551
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56,0.774418,0.479076,0.194155,3.052398,8.743455,0.860284,-0.088571,1.774601,6.560771,1.248051,...,2.674486,0.286614,0.598837,1.669903,1 days 05:48:00,172,103,205,3.604340,43.252075
14,0.794133,0.490273,0.177318,3.387426,11.773028,0.882910,-0.067454,1.925028,7.577172,1.250427,...,2.067292,0.283885,0.616279,1.622642,1 days 04:58:00,172,106,211,3.673996,44.087949
36,0.794643,0.490562,0.242639,2.531798,9.426845,0.867851,-0.084296,1.632458,5.338108,1.294614,...,2.415486,0.277420,0.593023,1.686275,1 days 07:06:00,172,102,204,3.454847,41.458166
83,0.800791,0.497537,0.192886,3.147659,8.888846,0.911741,-0.090089,1.889628,7.071066,1.574671,...,2.292662,0.276900,0.583815,1.712871,1 days 07:18:00,173,101,202,3.394823,40.737871


In [11]:
res['Sharpe ratio'].median()

1.0820878364216704

In [12]:
res['Sharpe ratio'].mean()

1.0350027430526523

In [13]:
res[res['Annual return'] >=.2]['Annual return'].count()

52

In [14]:
res['Position EV'].median()

0.07867549875957108

In [15]:
res['Position EV'].mean()

0.08389023310003733

In [16]:
(res[res['Position EV'] < 2])['Annual return'].count()

100

In [17]:
(res[res['Position EV'] < 5])['Annual return'].count()

100

In [18]:
res['Monthly EV'].median()

1.082550344589817

In [19]:
res['Monthly EV'].mean()

1.0772958766343255